<a href="https://colab.research.google.com/github/JOSEPHINEGEND/Sendy-Logistics-Cape-to-Nairo-Datathon/blob/master/We_are_the_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### R Insights
- CRISP-DM
    - Business Understanding
    - Data Understanding
    - Data Preparation
    - Modelling
    - Deployment
    
### Research Question

---


Predict the time for delivery

### Libraries to use
 - pacman
 - tidyverse
 - caret
 - skimr
 
### Data Cleaning
 - Rename columns - Using the gsub function
 - Remove white  spaces
 - Encoding categorical variables
   - Check for distribution of categorical variables
   - 
 
### Feature Wngineering
 - Data accuracy
 - Create features from time
 - Distribution of the target variable
 - Normalise the data
 - Check for anomalies
   - Missing data
     - Precipitation and Temperature 
     - Can impute using
      - Mean
   - Outliers
     - Impute
     - Drop
     - Create a new column with True False to reflect whether there was an outlier
   - Skewed distributions
     - Investigate the zero delivery time
     - Set a threshold of two minutes or more
 - Check for measures of measures of central tendacy
  - Normalise variables
  
### Bivariate analysis
 - Temperature and distance have a postive relationship
 #### Variable Selction
 **
 
### Modelling
 - Bias and variance tradeoff
 - Shuffle the data
 - Create train, test and validation sets
 - Regression models
 
### Cross-validation
 - compare crossval errors with submission
 
### Parameter tuning
 - Randomsearch
 - Gridsearch
 
### Metrics of Success
 - RME - Most common
 - MAE - Robust with data that has outliers
 #### Try different models and compare the rmse's
 
## Python Insights
 - Hacking
  - MLXNET
  - Blending
  - mirror/mean encoding
  
  

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import requests
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LinearRegression
from io import StringIO
import lightgbm as lgb
import lightgbm as lgbb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import hyperopt
import datetime
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [0]:
!pip install catboost

     |████████████████████████████████| 62.9MB 114kB/s 


In [0]:
from catboost import CatBoostRegressor

In [0]:
train = 'https://drive.google.com/file/d/1ZP9pFBATu38l97Tut5hKLvpzKRLFXX_P/view?usp=sharing'
test = 'https://drive.google.com/file/d/1_aElMoEIRs55avOafA7U1_YXEuaDBXLh/view?usp=sharing'
submission = 'https://drive.google.com/file/d/1mqXS8euMqF9_bhTEU6O9cLHoX2FI_5HD/view?usp=sharing'
dictionary = 'https://drive.google.com/file/d/1juqltwSs6OXQgJJEhTxD7Gm443fnLpCp/view?usp=sharing'
riders = 'https://drive.google.com/file/d/19-aVgAcKRxX_Tk9StUQMNeAUVi0ZTo9K/view?usp=sharing'

def read_csv(url):
  url = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
  csv_raw = requests.get(url).text
  csv = StringIO(csv_raw)
  return csv

train = pd.read_csv(read_csv(train))
tes = pd.read_csv(read_csv(test))
sub = pd.read_csv(read_csv(submission))
dictionary = pd.read_csv(read_csv(dictionary))
riders = pd.read_csv(read_csv(riders))

### Cleaning column names

In [0]:
df = train.copy()
test = tes.copy()
y = train[['User Id', 'Time from Pickup to Arrival']]
df, test = df.align(test, join = 'inner', axis = 1)


df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('=', '_')
df.columns = df.columns.str.replace('__', '_')
df.columns = df.columns.str.replace('(', '').str.replace(')', '')
df.columns = df.columns.str.replace('__', '_')

test.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('=', '_')
test.columns = df.columns.str.replace('__', '_')
test.columns = df.columns.str.replace('(', '').str.replace(')', '')
test.columns = df.columns.str.replace('__', '_')

df['separator'] = 1
test['separator'] = 0

tt = pd.concat([df, test])

riders.columns = riders.columns.str.lower().str.replace(' ', '_')
comb = tt.merge(riders, how = 'left', on = 'rider_id')

#Stripping the order no to have whole numbers
comb['order_no']= comb['order_no'].str.replace('Order_No_', '')
comb['order_no']=comb['order_no'].astype(int)
comb.sort_values(by=['order_no'], ascending= True)


comb['temp'] = comb['temperature'].apply(lambda x: True if x == np.nan else False)
#Forward Fill Temprature Column
comb['temperature'].fillna(df.temperature.mean(), inplace= True)

#Backward Fill Temprature column
#comb['temperature'].fillna(method='bfill', inplace= True)

#Filling in the Precipitation column with 0
comb['precipitation_in_millimeters'].fillna(0, inplace= True)

time_cols = ['placement_time', 'confirmation_time', 'pickup_time', 'arrival_at_pickup_time']
for col in time_cols:
  comb[col] = pd.to_datetime(comb[col])
  comb[col.split('_')[0] + '_hour'] = comb[col].dt.hour
  #comb[col.split('_')[0] + '_minute'] = comb[col].dt.minute
  #comb[col] = pd.to_datetime(comb[col], format = '%H:%M:%S', errors = 'coerce')
  #comb[col] = [time.time() for time in comb[col]]
  
  
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 15, init ='k-means++')
kmeans.fit(comb[['pickup_lat', 'pickup_long']]) # Compute k-means clustering.
comb['pickup_cluster_label'] = kmeans.fit_predict(comb[['pickup_lat', 'pickup_long']])
centers1 = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels1 = kmeans.predict(comb[['pickup_lat', 'pickup_long']]) # Labels of each point
comb['pickup_cluster_label'] = comb['pickup_cluster_label'].astype('category')
#comb = comb.drop(columns=['pickup_lat', 'pickup_long'])
kmeans = KMeans(n_clusters = 15, init ='k-means++')
kmeans.fit(comb[['destination_lat', 'destination_long']]) # Compute k-means clustering.
comb['destination_cluster_label'] = kmeans.fit_predict(comb[['destination_lat', 'destination_long']])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(comb[['destination_lat', 'destination_long']]) # Labels of each point
comb['destination_cluster_label'] =comb['destination_cluster_label'].astype('category')
#comb = comb.drop(columns=['destination_lat', 'destination_long'])


#Peak offpeak 7 - 10 or 4 - 8
hour_cols = ['placement_hour',	'confirmation_hour',	'pickup_hour', 'arrival_hour']
for col in hour_cols:
  comb[col.split('_')[0] + '_peak'] = comb[col].apply(lambda x: 1 if ((x >= 7) & (x <= 9)) | ((x >= 16) & (x <= 19)) else 0)
  
time_cols = ['placement_time', 'confirmation_time', 'pickup_time']
for col in time_cols:
  comb['arrival_minus_' + col.split('_')[0]] = (comb.arrival_at_pickup_time - comb[col]).astype('timedelta64[s]')
  
time_cols = ['placement_time', 'confirmation_time']
for col in time_cols:
  comb['pickup_minus_' + col.split('_')[0]] = (comb.pickup_time - comb[col]).astype('timedelta64[s]')
  
comb['confir_minus_placement'] = (comb.confirmation_time - comb.placement_time).astype('timedelta64[s]')

cat_cols = ['platform_type', 'personal_or_business', 'placement_day_of_month',
       'placement_weekday_mo_1', 'placement_hour',
       'confirmation_hour', 'pickup_hour', 'arrival_hour',
       'pickup_cluster_label', 'destination_cluster_label', 'placement_peak',
       'confirmation_peak', 'pickup_peak', 'arrival_peak']

for col in cat_cols:
  comb[col] = comb[col].astype('category')

cols_drop = ['order_no', 'user_id', 'vehicle_type', 'placement_time', 'confirmation_day_of_month',
       'confirmation_weekday_mo_1', 'confirmation_time',
       'arrival_at_pickup_day_of_month', 'arrival_at_pickup_weekday_mo_1',
       'arrival_at_pickup_time', 'pickup_day_of_month', 'pickup_weekday_mo_1',
       'pickup_time', 'rider_id']


comb.drop(cols_drop, axis = 1, inplace = True)

train = comb[comb.separator == 1]
test = comb[comb.separator == 0]

train.drop('separator', axis = 1, inplace = True)
test.drop('separator', axis = 1, inplace = True)

train['y'] = y['Time from Pickup to Arrival']

In [0]:
tra = train.copy()
tes = test.copy()

In [0]:
y = tra.y
tra = pd.get_dummies(tra)
X = tra.drop('y', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

cat = CatBoostRegressor()
cat.fit(X_train, y_train)

y_pred = cat.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))

tes = pd.get_dummies(tes)

pred = cat.predict(tes)

sub_pred = pd.DataFrame({"Order_No":sub['Order_No'], 
                           "Time from Pickup to Arrival": pred })
sub_pred.to_csv("111.csv", index=False)